In [22]:
import torch
import torch.nn as nn
from torch.nn import functional as F

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4

max_iters = 1000
learning_rate = 3e-4
eval_iters = 250

cuda


In [2]:
with open('wizard-of-oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()

chars = sorted(set(text))
print(chars)

vocab_size = len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


In [15]:
string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([80,  1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,
         1, 47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26,
        49,  0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,
         0,  0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1,
        36, 25, 38, 28,  1, 39, 30,  1, 39, 50])


In [8]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    # print(ix)
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs:')
# print(x.shape)
print(x)


print('targets:')
print(y)

tensor([ 6251, 46257, 80641, 42710])
inputs:
tensor([[54,  1, 69, 54, 71, 54, 72, 68],
        [73, 72,  1, 55, 58, 73, 73, 58],
        [65, 68, 71, 58, 57,  1, 72, 74],
        [ 1, 76, 54, 73, 56, 61, 58, 57]], device='cuda:0')
targets:
tensor([[ 1, 69, 54, 71, 54, 72, 68, 65],
        [72,  1, 55, 58, 73, 73, 58, 71],
        [68, 71, 58, 57,  1, 72, 74, 67],
        [76, 54, 73, 56, 61, 58, 57,  1]], device='cuda:0')


In [6]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [18]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)
        
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


KI]t0z
5wt:9FVGwCHDj]8)H.kO9Au)1_7:shf1&VGgcYZzSn CX1&v.)Az7E99S[Zj-IJHTF!"D*Ic6TPR9kMr(RKoc
SJ'L!'Lko?sM;MkmBU;tnB47bsNTeFBaBIl_a 45c3G1﻿Z27]qE6]U1eRbwK-[!s5PT.) -J1;Oax""sM E]]5?HNsEwR)L2"wE"5e nIn0&u;:Lkd6B[ODCTPg
H'T?I5A twVZvryP0wieC I?"zhXYQf85CThAvkIsw﻿Z6.;qWLqvW﻿ZgIT﻿cf1(S)_I5m*
513 7x6f1,v7w-D_D1zI_2x
CF(K3 XH!5p15;BioBRu-z]]olPol[IexR! EBujlS: ooX
SwNTJS﻿4BZvkmBZTG5u-H*w.N-YDOnl-RS&2ETT*I(ieB)1Oa"5t&T!YlirKfpIP﻿,4a(]qA IQdbFENM9BR*.4sUk
HceYPTSf4DpRxt85;o"qAomK-Xp*9n?IxuI[[gc_BKq]q]'ME


In [23]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

tensor([ 68401,  19367,  39831, 131924])
tensor([153530, 106005, 180876, 181847])
tensor([162963, 179411,  18154, 108911])
tensor([120529,  21904,  64290, 177228])
tensor([117365,  91978, 149222,  81017])
tensor([178306, 133261, 152797, 153261])
tensor([ 97557, 169765,  18726, 163793])
tensor([ 76614,  71902, 129585, 127150])
tensor([50696, 24008,  5589, 17978])
tensor([178390, 176914, 144069, 155368])
tensor([ 38963, 146008,  87317, 174034])
tensor([ 99733, 104638, 140663, 107858])
tensor([167775,  39072,  14398, 122492])
tensor([162711, 171515, 143351,  50618])
tensor([130846,  77618,  10169, 174093])
tensor([147279, 124789, 175952,  87083])
tensor([ 11759, 180122, 125402,  31230])
tensor([  7430,  42332,  24744, 117237])
tensor([163616,  33496, 172536,   9993])
tensor([127764,  14979, 126103, 109836])
tensor([ 52207,  73813, 111342,  57568])
tensor([155075,  53775, 151654, 132911])
tensor([ 77144, 134314, 155687, 115895])
tensor([175391,  91619, 136697,  61552])
tensor([ 92256,  667

In [26]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


&VuTs,ZNp.)]?pwI
-C.)1wJ﻿2ivs9: Z7﻿;W!;,bF]I4WND1ARGnd"Ht8? *ICE
;P?,q0:JE85"?s28WduzPq'd(PTM;"UxEK-GoS3 zMD0"!w1F ﻿9-ycf&x; ol(.f*w[Ot18jB-IEp9g-GzLaoBKF*7Ee_XOD-cYsB5;zSXpz.pR X'Z
]([DioGd'NTl1[Yd1'p&*wJ!zAzJ'94ag0:-]MU1f13uTWotq
;qFrzAZ,ssTB
;Sl)u&uG]]HCuc"!(g1Um?sVY.
TWK4d?6t;a UDnZ(iNJQ1kC6&3,KAE;6&l)9KqhV&o.pYgIgz1:l,]10M_h:S-m[hOJyFJW(OvSYi;9gEP)06T*Q3(4*
dw:z(]Inl"EwIQJ﻿"zL2_FxW:[Y[IBE?gDZ(:]oeV&uhJnZUx8xhnBpmtx,Kja﻿92_By"D1agbnxaol1EusVLk?3UzVY﻿d"﻿P?p1p_bA( :PGsUs'by3LkolMU*nZz1p1!Pqj]d
